In [ ]:
from my_weapon import *
from tqdm import tqdm_notebook as tqdm
sns.set(style="whitegrid", font_scale=1.3)
# plt.style.use('fivethirtyeight')

In [ ]:
def definir_ponderadores_rake(dt_imagen, variable):
    if variable != "":
        dt_imagen = dt_imagen[dt_imagen[variable].notnull()]
        
    if 'pondera_base' not in dt_imagen:
        dt_imagen['pondera_base'] = 1

    for x in range(0, 10):
        for p_id, p_info in rake.items():
            print("campo:", p_id)
            dt_imagen_agg = dt_imagen.groupby(['semana2', p_id]).agg({'pondera_base': 'sum'})
            # print(dt_imagen_agg)
            share_semana = dt_imagen_agg.groupby(level=0).apply(lambda x: x / float(x.sum()))
            # print(share_semana)
            share_semana = share_semana.reset_index().rename(columns = {'pondera_base': 'share_base'})

            dt_imagen = pd.merge(dt_imagen, share_semana,  how='left', left_on=['semana2',p_id], right_on = ['semana2',p_id])

            dt_imagen['target_share'] = 0
            for key in p_info:
                print(str(key) + ':', p_info[key])    
                dt_imagen['target_share'].loc[dt_imagen[p_id] == key] = p_info[key]

            dt_imagen['pondera_base'] =  dt_imagen['pondera_base'] / dt_imagen['share_base'] * dt_imagen['target_share']
            # print(dt_imagen)
            
            del dt_imagen['share_base']
            del dt_imagen['target_share']

    return dt_imagen

In [ ]:
# rake = {
#             'Usted es ': {'Hombre': 0.479, 'Mujer': 0.521},
#             'Edad ':{'Entre 16 y 30': 0.311, 'entre 31 y 50': 0.323, 'entre 51 y 65': 0.193, 'Mas de 65 años': 0.136},
#             '¿Cuál es su nivel de estudio? ':{'Secundario Incompleto': 0.537, 'Secundario completo':0.208, 'Univ. completo o incompleto':0.255},
#             'polarizado': {True: 0.72, False: 0.28}
# }

rake = {
    'Usted es ': {'Hombre': 0.479, 'Mujer': 0.521},
    'Edad ':{'Entre 16 y 30': 0.311, 'entre 31 y 50': 0.323, 'entre 51 y 65': 0.193, 'Mas de 65 años': 0.136},
    '¿Cuál es su nivel de estudio? ': {'Secundario Incompleto': 0.537, 'Secundario completo':0.208, 'Univ. completo o incompleto':0.255} # 文化水平
}

# 性别，年龄，教育程度迭代
# keep location firstly
df_intencion['semana2'] = df_intencion['¿En qué provincia vota? '] # Location
df_intencion_n = definir_ponderadores_rake(df_intencion, '')

# 位置迭代
df_intencion_n['semana2'] = 1
rake = { '¿En qué provincia vota? ': {'Int. PBA': 0.13, 'Resto': 0.55, 'GBA': 0.24, 'CABA': 0.08} }
df_intencion_n = definir_ponderadores_rake(df_intencion_n, '')
df_intencion_n.head()